In [3]:
import botometer
import pandas as pd

In [5]:
mashape_key = '672271137cmsh2f76f749d6aaaffp14476fjsn1c5eb01bcb0d'

twitter_app_auth = {
    'consumer_key': 'mzfgwXkTwLlxYt0qqg2KZNpZ6',
    'consumer_secret': 'pFCHkAxDqMyn1gkMoLKlWPG9Cc2OQZOVuYzuzvHBuG1SNLeCqS',
    'access_token': '39675338-JgDKqr7hMyaO5bp7rGgM2ADfuwk6Oz9rkx8Sn2UpK',
    'access_token_secret': 'mrkybSdUIx1MjxgmGeVUYW93ggSXD5S3d0BAunMlNvkzw',
  }


bom = botometer.Botometer(wait_on_ratelimit=True,
                          mashape_key=mashape_key,
                          **twitter_app_auth)

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("/Users/andresgf91/Capstone/Data/SQL_META_HARVEY.sqlite")
all_ids = pd.read_sql_query("select user_id_str from all_texas_tweets;", conn)
all_ids = set(all_ids['user_id_str'])

print('finished getting business from  SQL')

all_ids = list(set(all_ids))
len(all_ids)

In [ ]:
prev_results = pd.read_csv(filepath_or_buffer="~/Capstone/Data/botometer_2.csv")

prev_results.head(10)

prev_ids = list(set(prev_results['ids']))
len(prev_ids)
all_ids = list(set(all_ids))

all_ids = [int(i) for i in all_ids]

In [ ]:
still_needs_searching = [i for i in all_ids if i not in prev_ids]
print(len(still_needs_searching))

In [58]:
to_search = still_needs_searching[0:400]

In [ ]:
ls = list()
accounts = to_search
for screen_name, result in bom.check_accounts_in(accounts):
    ls.append(result)
    
print('Finished running botometer scores...')
len(ls)

In [ ]:
import pandas as pd

old_botometer_df = pd.read_csv(filepath_or_buffer="~/Capstone/Data/botometer_2.csv")

try:
    del old_botometer_df['Unnamed: 0']
except KeyError: 
    pass

errors = list()

for i in range(0,len(ls)):
    try:
        d=ls[i]['user']
        d=ls[i]['scores']
    except KeyError:
        errors.append(i)
        
ls = [ls[i] for i in range(0,len(ls)) if i not in errors]

ids = list(ls[i]['user']['id_str'] for i in range(0,len(ls)))
english_score = list(ls[i]['scores']['english'] for i in range(0,len(ls)))
universal_score = list(ls[i]['scores']['universal'] for i in range(0,len(ls)))

new_botometer_df = pd.DataFrame({'ids':ids,'english_score':english_score,'universal_score':universal_score})

botometer_df = new_botometer_df.append(old_botometer_df, ignore_index = True, sort=False)
botometer_df = pd.DataFrame.drop_duplicates(botometer_df,subset='ids')

pd.DataFrame.to_csv(botometer_df,path_or_buf="~/Capstone/Data/botometer_2.csv", index= False)

len(botometer_df)